In [48]:
import pandas as pd
from server_config import ENGINE_LINK
from sqlalchemy import create_engine

import re

from pymorphy2 import MorphAnalyzer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('wordnet')

from sklearn.feature_extraction.text import TfidfVectorizer

from joblib import Parallel, delayed

from tqdm import tqdm

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\skliz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\skliz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [49]:
with create_engine(ENGINE_LINK).connect() as conn:
    df = pd.read_sql("select * from tvs where total_duration notnull", conn)
    
df.tail()

,type,title_ru,title_eng,description,year_start,year_end,genres,myshows_rating,myshows_num,kp_rating,kp_num,imdb_rating,imdb_num,country,total_duration,series_duration,seasons,production,poster,link
176440,Сериал,На замену,The Replacements,"Тодд и Райли жили в приюте, не зная, что случи...",2006,2009,"Семейный,Детский,Приключения",3.74,300,5.829,1317,5.8,2983,США,1 день 3 часа 30 минут,22 мин.,2,Disney Channel,https://media.myshows.me/shows/1920/2/24/22458...,https://myshows.me/view/13300/
176441,Сериал,Срок,Time,Марк Кобден осужден на четыре года за неосторо...,2021,2021,"Драма,Триллер",4.05,1112,7.557,29158,8.2,20416,Великобритания,3 часа 2 минуты,60 мин.,1,BBC One,https://media.myshows.me/shows/1920/2/df/2dfab...,https://myshows.me/view/73329/
176442,Сериал,Убийство первой степени,Murder in the First,"В центре сюжета сериала, действие которого раз...",2014,2016,"Криминал,Драма,Детектив",3.51,1173,6.595,1627,7.2,7301,США,22 часа 56 минут,43 мин.,3,TNT DRAMA,https://media.myshows.me/shows/1920/f/2c/f2cf9...,https://myshows.me/view/35641/
176443,Сериал,Борджиа,Borgia,Шокирующий путь восхождения к власти и доминир...,2011,2014,"Приключения,Драма,Исторический",4.04,1140,7.863,25408,7.8,6721,Франция,1 день 12 часов 6 минут,57 мин.,3,Canal+,https://media.myshows.me/shows/1920/d/4d/d4d04...,https://myshows.me/view/21508/
176444,Сериал,Самый лучший хит,The Best Hit,"Юн Хён Дже (Юн Ши Юн) — музыкальный гений, уча...",2017,2017,"Драма,Музыкальный,Романтика",4.32,1394,7.781,2905,7.6,764,"Южная,Корея",16 часов,30 мин.,1,KBS2,https://media.myshows.me/shows/1920/6/57/65785...,https://myshows.me/view/53621/


In [50]:
df['type'] = df['type'].map({'Фильм' : 'film', 'Аниме' : 'series', 'Мультфильм' : 'film', 'Мультсериал' : 'series', 'Сериал' : 'series'})

In [51]:
def transform_duration(x):
    x = re.sub(' +', '', x)
    
    res = 0
    
    TIME = {
        'мес' : 720, # 30 дней
        'нед' : 168,
        'день' : 24,
        'дня': 24,
        'дней' : 24,
        'час' : 1,
        'мин' : 1/60
    }
    
    for key in TIME.keys():
        try:
            res += (int(re.search(r'\d+', re.search(fr'\d+{key}', x).group(0)).group(0)) * TIME[key])
        except: pass
        
    return res

In [52]:
df['total_duration'] = df['total_duration'].map(transform_duration)

In [53]:
df['total_duration'].describe()

count    176445.000000
mean          9.268295
std          80.468568
min           0.000000
25%           1.250000
50%           1.583333
75%           2.433333
max        7598.316667
Name: total_duration, dtype: float64

In [54]:
df = df.loc[(df['year_start'].isna()==False) & (df['title_ru'].isna()==False) & (df['description'].isna() == False)].reset_index(drop=True)

In [55]:
def clean_text(x: str,
               morph: MorphAnalyzer = MorphAnalyzer(),
               sno: SnowballStemmer = SnowballStemmer('russian'),
               stop_words: list = stopwords.words('russian'),
               min_len_word: int = 2) -> str:
    x = x.lower()                      # Приведение к нижнему регистру
    x = re.sub(r'[^а-я]+', ' ', x)     # Удаление всех символов, цифр
    x = re.sub(r' +', ' ', x)          # Удаление повторяющихся пробелов

    word_tokenization = x.split(' ')   # Токенизация слов

    result_sentance = []

    # Приведение слов к нормальной форме (леммер + стеммер для причастий)
    for word in word_tokenization:
        if 'PRTF' in morph.parse(word.strip())[0].tag or 'PRTS' in morph.parse(word.strip())[0].tag:
            result_word = sno.stem(word)
        else:
            result_word = morph.parse(word.strip())[0].normal_form

        if result_word not in stop_words and len(result_word) > min_len_word:
            result_sentance.append(result_word)

    return ' '.join(result_sentance)

In [56]:
clean_description = Parallel(n_jobs=-1)(delayed(clean_text)(t) for t in tqdm(df['description']))
df['clean_description'] = clean_description

100%|██████████| 69505/69505 [15:33<00:00, 74.47it/s]


In [57]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))

X = vectorizer.fit_transform(df['clean_description'])

In [58]:
df['vectorizer'] = list(X)

In [59]:
df['show_id'] = df['title_ru'] + '_' + df['year_start'] + '_' + df['type']

In [60]:
df[['show_id', 'total_duration', 'kp_rating', 'imdb_rating', 'genres', 'clean_description', 'vectorizer']]

,show_id,total_duration,kp_rating,imdb_rating,genres,clean_description,vectorizer
0,Вставай! Оторвись!_2020_series,1.000000,None,None,"Аниме,Комедия",история творческий трение ревность среди юный ...,"(0, 1449085)\t0.2874276844462563\n (0, 5490..."
1,У нашего старшего брата проблемы с головой_201...,3.583333,None,None,"Боевик,Приключения,Аниме,Комедия",приключение четыре ученик путь совершенствован...,"(0, 1452215)\t0.2840117755195547\n (0, 5908..."
2,Супер братья Сейшин_2013_series,0.933333,None,None,"Комедия,Аниме",история вращаться вокруг сестра младший брат ч...,"(0, 720987)\t0.17603081763783227\n (0, 4103..."
3,Стальные Хроники Царств_2007_series,9.166667,None,None,"Приключения,Аниме,Исторический",существовать печать который должный носить изб...,"(0, 1060006)\t0.15623172751978565\n (0, 589..."
4,Метка дракона_2014_series,5.000000,5.995,6.6,"Аниме,Романтика,Фэнтези,Комедия,Боевик",посреди могучий держава лежать небольшой горны...,"(0, 360289)\t0.05407970629428435\n (0, 6944..."
...,...,...,...,...,...,...,...
69500,На замену_2006_series,27.500000,5.829,5.8,"Семейный,Детский,Приключения",тодда райли жить приют знать случиться биологи...,"(0, 427515)\t0.11044447762839461\n (0, 1660..."
69501,Срок_2021_series,3.033333,7.557,8.2,"Драма,Триллер",марк кобден осужд четыре год неосторожный убий...,"(0, 1115724)\t0.1529117029917302\n (0, 8783..."
69502,Убийство первой степени_2014_series,22.933333,6.595,7.2,"Криминал,Драма,Детектив",центр сюжет сериал действие который разворачив...,"(0, 1655193)\t0.1693229607495187\n (0, 1464..."
69503,Борджиа_2011_series,36.100000,7.863,7.8,"Приключения,Драма,Исторический",шокир путь восхождение власть доминирование са...,"(0, 944170)\t0.12858310246141283\n (0, 3914..."


In [61]:
df[['show_id', 'total_duration', 'kp_rating', 'imdb_rating', 'genres', 'clean_description', 'vectorizer']].to_csv('data/tvs_bd.csv', index=False)